In [1]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import albumentations as A
import cv2
import os

In [2]:
json_path = './dtag_rectangle_20241126/dtag_export/annotations/instances_val.json'
imgs_path = './dtag_rectangle_20241126/dtag_export/val/'
with open(json_path, 'r') as file:
    imgs_json_data = json.load(file)

In [3]:
original_num_imgs_folder = len(imgs_json_data["images"])

In [4]:
last_id_num = imgs_json_data["images"][-1]["id"]

In [5]:
last_id_num_annotations = last_id_num

In [6]:
last_id_num_annotations

250

In [7]:
print(last_id_num)

250


In [8]:
len(imgs_json_data["images"])

58

In [9]:
len(imgs_json_data["annotations"])

76

In [10]:
transformations = {
    "brightness_contrast_blur" : A.Compose([
    A.RandomBrightnessContrast(p=0.7),
    A.Blur(blur_limit=(3,7), p=1.0),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.5)
    ]),
    "grayscale_noise": A.Compose([
        A.ToGray(p=1.0),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.8)
    ]),
    "GRAY_noise": A.Compose([
        A.ToGray(p=1.0),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.8)
    ]),
    "NOISE_gray": A.Compose([
        A.ToGray(p=1.0),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.8)
    ]),
}

"grayscale_noise": A.Compose([
        A.ToGray(p=1.0),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.8)
    ]),

In [11]:
print(imgs_json_data["images"][25])

{'license': 'NA', 'file_name': 'c6c89ac7-dfd4-4acc-9b74-c43d2769e09c.jpeg', 'height': 576, 'width': 768, 'date_captured': '2024-08-26 13:38:51.713000+00:00', 'id': 218}


In [12]:
next(iter(transformations))

'brightness_contrast_blur'

In [13]:
for transformation in transformations:
    print(transformation)


brightness_contrast_blur
grayscale_noise
GRAY_noise
NOISE_gray


In [14]:
def find_times_id_repeated_annotations(annotations_list, id_num):
    counter = 0
    positions = []
    for item in range(len(annotations_list)):
        #print(annotations_list[item])
        if annotations_list[item]["image_id"] == id_num:
            counter += 1
            #print("eureka")
            positions.append(item)
    return positions

In [15]:
find_times_id_repeated_annotations(imgs_json_data["annotations"], 275)

[]

In [16]:
len(imgs_json_data["annotations"])

76

for img in range(original_num_imgs_folder):
    #print(imgs_json_data["images"][img])
    file_img_name = imgs_json_data["images"][img]["file_name"]
    full_path = os.path.join(imgs_path, file_img_name)
    print(full_path)
    image = cv2.imread(full_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for transformation_name, transformation in transformations.items():
        transformed_image = transformation(image=image)
        name, ext = os.path.splitext(file_img_name)
        file_transformed_name = f"{name}_{transformation_name}{ext}"
        transformed_fullfile_path = os.path.join(imgs_path, file_transformed_name)
        print(transformed_fullfile_path)
        print(imgs_json_data["images"][img]["id"])
        imgs_json_data["images"].append({
            "license": "NA",
            "file_name": file_transformed_name,
            "height": 576,
            "width": 768,
            "date_captured": imgs_json_data["images"][img]["date_captured"],
            "id": last_id_num + 1 
        })
        #cv2.imwrite(transformed_fullfile_path, transformed_image["image"])
        temp_locations = find_times_id_repeated_annotations(imgs_json_data["annotations"], imgs_json_data["images"][img]["id"])
        """if len(temp_locations) > 0:
            for item in temp_locations:
                temp_dict = imgs_json_data["annotations"][item]
                temp_dict["image_id"] = last_id_num_annotations + 1
                imgs_json_data["annotations"].append(temp_dict)
            last_id_num_annotations += 2
        else:
            last_id_num_annotations += 1"""
        
        
        
        """for item in range(len(imgs_json_data["annotations"])):
            if imgs_json_data["annotations"][item]["image_id"] == imgs_json_data["images"][img]["id"]:
                print("eureka")
                temp_dict = imgs_json_data["annotations"][item]
                temp_dict["image_id"] = last_id_num_annotations + 1
                imgs_json_data["annotations"].append(temp_dict)
                last_id_num_annotations += 1"""
        
        last_id_num += 1

        if len(temp_locations) > 0:
            counter = 0
            for item in temp_locations:
                temp_dict = imgs_json_data["annotations"][item]
                print(temp_dict)
                temp_dict["image_id"] = counter + last_id_num
                print(temp_dict)
            counter =+ 1
        
        
        print(last_id_num)

In [17]:
for img in range(original_num_imgs_folder):
    file_img_name = imgs_json_data["images"][img]["file_name"]
    full_path = os.path.join(imgs_path, file_img_name)
    image = cv2.imread(full_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    original_image_id = imgs_json_data["images"][img]["id"] 
    
    for transformation_name, transformation in transformations.items():
        transformed_image = transformation(image=image)
        
        
        name, ext = os.path.splitext(file_img_name)
        file_transformed_name = f"{name}_{transformation_name}{ext}"
        transformed_fullfile_path = os.path.join(imgs_path, file_transformed_name)
        print(transformed_fullfile_path)

        new_image_id = last_id_num + 1
        imgs_json_data["images"].append({
            "license": "NA",
            "file_name": file_transformed_name,
            "height": 576,  
            "width": 768,   
            "date_captured": imgs_json_data["images"][img]["date_captured"],
            "id": new_image_id
        })
        
        temp_locations = find_times_id_repeated_annotations(
            imgs_json_data["annotations"], original_image_id
        )
        
        if len(temp_locations) > 0:
            for item in temp_locations:
                temp_dict = imgs_json_data["annotations"][item].copy()
                temp_dict["image_id"] = new_image_id  
                imgs_json_data["annotations"].append(temp_dict)  
        
        last_id_num += 1
        print(last_id_num)


./dtag_rectangle_20241126/dtag_export/val/b35bcc83-e8f4-42b7-8b49-f973eda5ae61_brightness_contrast_blur.jpeg
251
./dtag_rectangle_20241126/dtag_export/val/b35bcc83-e8f4-42b7-8b49-f973eda5ae61_grayscale_noise.jpeg
252
./dtag_rectangle_20241126/dtag_export/val/b35bcc83-e8f4-42b7-8b49-f973eda5ae61_GRAY_noise.jpeg
253
./dtag_rectangle_20241126/dtag_export/val/b35bcc83-e8f4-42b7-8b49-f973eda5ae61_NOISE_gray.jpeg
254
./dtag_rectangle_20241126/dtag_export/val/b425b23e-1f18-499c-926a-f671fc761c3d_brightness_contrast_blur.jpeg
255
./dtag_rectangle_20241126/dtag_export/val/b425b23e-1f18-499c-926a-f671fc761c3d_grayscale_noise.jpeg
256
./dtag_rectangle_20241126/dtag_export/val/b425b23e-1f18-499c-926a-f671fc761c3d_GRAY_noise.jpeg
257
./dtag_rectangle_20241126/dtag_export/val/b425b23e-1f18-499c-926a-f671fc761c3d_NOISE_gray.jpeg
258
./dtag_rectangle_20241126/dtag_export/val/b5374744-58cd-4a45-b7b4-2092891ee75e_brightness_contrast_blur.jpeg
259
./dtag_rectangle_20241126/dtag_export/val/b5374744-58cd-4

In [18]:
imgs_json_data["images"]

[{'license': 'NA',
  'file_name': 'b35bcc83-e8f4-42b7-8b49-f973eda5ae61.jpeg',
  'height': 576,
  'width': 768,
  'date_captured': '2024-08-28 12:12:28.970000+00:00',
  'id': 193},
 {'license': 'NA',
  'file_name': 'b425b23e-1f18-499c-926a-f671fc761c3d.jpeg',
  'height': 576,
  'width': 768,
  'date_captured': '2024-08-26 13:38:50.867000+00:00',
  'id': 194},
 {'license': 'NA',
  'file_name': 'b5374744-58cd-4a45-b7b4-2092891ee75e.jpeg',
  'height': 576,
  'width': 768,
  'date_captured': '2024-08-28 12:12:27.187000+00:00',
  'id': 195},
 {'license': 'NA',
  'file_name': 'b54c4671-e522-4e98-9581-3094112637e2.jpeg',
  'height': 576,
  'width': 768,
  'date_captured': '2024-08-26 13:38:50.690000+00:00',
  'id': 196},
 {'license': 'NA',
  'file_name': 'b5e8707a-4b61-41bf-b53e-698117f924df.jpeg',
  'height': 576,
  'width': 768,
  'date_captured': '2024-08-28 12:12:26.607000+00:00',
  'id': 197},
 {'license': 'NA',
  'file_name': 'b6277351-7a8a-4f79-937b-6dfdb119f7cb.jpeg',
  'height': 576,

In [19]:
imgs_json_data["annotations"]

[{'segmentation': [[277.6615447998047,
    184.38816833496094,
    466.7076873779297,
    184.38816833496094,
    466.7076873779297,
    304.3597640991211,
    277.6615447998047,
    304.3597640991211]],
  'area': 22680,
  'iscrowd': 0,
  'image_id': 193,
  'id': 5395242014774222357,
  'bbox': [277, 184, 190, 121],
  'category_id': 1},
 {'segmentation': [[453.07456970214844,
    150.75976181030273,
    523.9668731689453,
    150.75976181030273,
    523.9668731689453,
    206.20117950439453,
    453.07456970214844,
    206.20117950439453]],
  'area': 3930,
  'iscrowd': 0,
  'image_id': 194,
  'id': 2517564716406979871,
  'bbox': [453, 150, 71, 57],
  'category_id': 2},
 {'segmentation': [[283.1147918701172,
    0.0,
    380.36448669433594,
    0.0,
    380.36448669433594,
    85.32071113586426,
    283.1147918701172,
    85.32071113586426]],
  'area': 8297,
  'iscrowd': 0,
  'image_id': 194,
  'id': 6425765092283219000,
  'bbox': [283, 0, 98, 86],
  'category_id': 1},
 {'segmentation': 

In [20]:
for item in range(len(imgs_json_data["annotations"])):
    if imgs_json_data["annotations"][item]["image_id"] == 260:
        print("eureka", item)

eureka 89


In [21]:
for item in range(len(imgs_json_data["annotations"])):
    dictionary = imgs_json_data["annotations"][item]
    for key, value in dictionary.items():
        if imgs_json_data["annotations"][item][key] == 260:
            print("eureka")

eureka
